In [57]:
import pandas as pd
import numpy as np

In [2]:
import lightgbm as lgb

In [48]:
lgb.__version__

'3.3.2'

In [70]:
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [17]:
df, y = load_diabetes(return_X_y=True, as_frame=True)

In [18]:
df['y'] = y

# まず連続値の予測
- 糖尿病患者の検査数値と1年後の進行状況のデータセット

In [64]:
target = 'y'
features = [c for c in df.columns if c != target]

In [65]:
train, test = train_test_split(df, test_size=0.2, random_state=0)
train, valid = train_test_split(train, test_size=0.2, random_state=0)

In [66]:
dtrain = lgb.Dataset(data=train[features], label=train[target], free_raw_data=False)
dtrain.construct()
dvalid = lgb.Dataset(data=valid[features], label=valid[target], free_raw_data=False)
dvalid.construct()

In [76]:
params = {
    'objective': 'regression',
    'boosting_type': 'gbdt',
    'learning_rate': 0.1,
    'max_depth': -1,
    'metric': 'root_mean_squared_error',
    'nthread': -1,
    'num_leaves': 7,
    'seed': 57,
    'verbose': -1,
    
#     'colsample_bytree': 0.05,
#     'min_child_weight': 10,
#     'min_split_gain': 0.1,
#     'reg_alpha': 10,
#     'reg_lambda': 1,
#     'subsample': 0.1,
}

In [77]:
model = lgb.train(
     params=params,
     train_set=dtrain,
     valid_sets=[dtrain, dvalid],
     num_boost_round=10000,
     early_stopping_rounds=10,
     verbose_eval=1,
)

[1]	training's rmse: 73.4856	valid_1's rmse: 78.4953
Training until validation scores don't improve for 10 rounds
[2]	training's rmse: 70.0847	valid_1's rmse: 75.6773
[3]	training's rmse: 66.7682	valid_1's rmse: 72.4375
[4]	training's rmse: 64.0703	valid_1's rmse: 70.2267
[5]	training's rmse: 61.5833	valid_1's rmse: 68.49
[6]	training's rmse: 59.4559	valid_1's rmse: 66.4946
[7]	training's rmse: 57.7101	valid_1's rmse: 65.2463
[8]	training's rmse: 56.0731	valid_1's rmse: 64.1842
[9]	training's rmse: 54.6497	valid_1's rmse: 62.9557
[10]	training's rmse: 53.2851	valid_1's rmse: 62.1439
[11]	training's rmse: 52.0745	valid_1's rmse: 61.7006
[12]	training's rmse: 51.0258	valid_1's rmse: 61.6334
[13]	training's rmse: 50.1159	valid_1's rmse: 61.426
[14]	training's rmse: 49.2769	valid_1's rmse: 60.9808
[15]	training's rmse: 48.559	valid_1's rmse: 61.0884
[16]	training's rmse: 47.7809	valid_1's rmse: 60.6058
[17]	training's rmse: 47.0865	valid_1's rmse: 60.6392
[18]	training's rmse: 46.4853	vali

/Users/yagikei/Workspace/github.com/keeeeei79/scratch_optimization/.venv/lib/python3.9/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/yagikei/Workspace/github.com/keeeeei79/scratch_optimization/.venv/lib/python3.9/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


In [78]:
mean_squared_error(test[target], model.predict(test[features]))**0.5

56.733714122137485

# Custom Objectiveを使って再現する
- Reference
    - https://www.f-denshi.com/000TokiwaJPN/10kaisk/040ksk.html
    - https://speakerdeck.com/rsakata/santander-product-recommendationfalseapurotitoxgboostfalsexiao-neta
    - https://hippocampus-garden.com/lgbm_custom/
    - https://yaakublog.com/lightgbm_custom

$L(y_{pred}+w, y) - L(y_{pred}, y)$を最小化するwを求める。　

一般にテイラー展開より以下が成り立つ。
$$
    f(x+h) = f(x) + \frac{f'(x)}{1!}h + \frac{f''(x)}{2!}h^2 + ......
$$
よって二次近似まで使うとすると
$$
    \Delta L = L(y_{pred}+w, y) - L(y_{pred}, y) \approx L'(y_{pred}, y)w + \frac{L''(y_{pred}, y)}{2}w^2
$$
この$\Delta L$の和を最小化するwを求める
$$
    f(w) = \sum_i^n (L'(y_{pred, i}, y_i)w + \frac{L''(y_{pred, i}, y_i)}{2}w^2) \\
    \frac{df(w)}{dw} = \sum_i^n (L'(y_{pred, i}, y_i) + L''(y_{pred, i}, y_i)w) = 0 \\
    w = - \frac{\sum_i^n L'(y_{pred, i}, y_i)}{\sum_i^n L''(y_{pred, i}, y_i)}
$$
ここで例えば$L(y_{pred}, y) = \frac{1}{2}(y_{pred}- y)^2$とすると 
$$
    L'(y_{pred}, y) = \frac{dL(y_{pred}, y)}{dy_{pred}} =  y_{pred}- y \\
    L''(y_{pred}, y) = \frac{d^2L(y_{pred}, y)}{dy_{pred}^2} = 1 \\
    (おそらくyは正解ラベルなので定数として扱っていい。本当はL(y_{pred})と表記した方がわかりやすいかも)
$$
以上より$L(y_{pred}, y)$についての1階微分と2階微分が分かれば損失を最小化するwがわかる。(本当はもう少し正則化項とかが入る)

GBDTではこのwを使って木を分岐した時に分岐しない時よりも損失が小さくなっていれば分岐を行うということを繰り返しモデルを構築する

In [79]:
def mse_loss(pred, data):
    label = data.get_label()
    grad = pred - label
    hess = np.ones_like(pred)
    return grad, hess

In [80]:
model = lgb.train(
     params=params,
     train_set=dtrain,
     valid_sets=[dtrain, dvalid],
     fobj=mse_loss,
     num_boost_round=10000,
     early_stopping_rounds=10,
     verbose_eval=1,
)

[1]	training's rmse: 151.635	valid_1's rmse: 170.931
Training until validation scores don't improve for 10 rounds
[2]	training's rmse: 138.427	valid_1's rmse: 157.107
[3]	training's rmse: 126.494	valid_1's rmse: 144.314
[4]	training's rmse: 115.994	valid_1's rmse: 133.34
[5]	training's rmse: 106.61	valid_1's rmse: 123.794
[6]	training's rmse: 98.3325	valid_1's rmse: 114.899
[7]	training's rmse: 91.1001	valid_1's rmse: 107.477
[8]	training's rmse: 84.6693	valid_1's rmse: 100.941
[9]	training's rmse: 79.0354	valid_1's rmse: 94.6975
[10]	training's rmse: 74.0264	valid_1's rmse: 89.4102
[11]	training's rmse: 69.6465	valid_1's rmse: 85.2989
[12]	training's rmse: 65.8493	valid_1's rmse: 81.7802
[13]	training's rmse: 62.5694	valid_1's rmse: 78.6893
[14]	training's rmse: 59.7069	valid_1's rmse: 75.868
[15]	training's rmse: 57.2599	valid_1's rmse: 73.7559
[16]	training's rmse: 55.0345	valid_1's rmse: 71.4946
[17]	training's rmse: 53.1152	valid_1's rmse: 69.9392
[18]	training's rmse: 51.48	valid

/Users/yagikei/Workspace/github.com/keeeeei79/scratch_optimization/.venv/lib/python3.9/site-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/Users/yagikei/Workspace/github.com/keeeeei79/scratch_optimization/.venv/lib/python3.9/site-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


In [81]:
mean_squared_error(test[target], model.predict(test[features]))**0.5

58.27303609853879

# 二値分類

In [ ]:
def original_init_score(y):
    y = y.mean()
    return np.log(y/(1-y))

In [ ]:
dtrain = lgb.Dataset(data=train[features], label=train[target], init_score=np.full_like(train[target], original_init_score(train[target]), dtype=float))
dvalid = lgb.Dataset(data=valid[features], label=valid[target], init_score=np.full_like(valid[target], original_init_score(train[target]), dtype=float))